## <center> ML-7. ПРОГНОЗИРОВАНИЕ БИОЛОГИЧЕСКОГО ОТВЕТА (HW-3)

In [154]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn')

from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

import warnings
warnings.filterwarnings("ignore")

In [155]:
data = pd.read_csv('_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [156]:
#Создаем матрицу наблюдений X и вектор ответов y
X = data.drop(['Activity'], axis=1)
y = data['Activity']

#Разделяем выборку на тренировочную и тестовую в соотношении 80/20. Для сохранения соотношений целевого признака используем параметр stratify (стратифицированное разбиение).
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

<center> РЕЗУЛЬТАТЫ БЕЗ ПРЕДВАРИТЕЛЬНЫХ НАСТРОЕК

In [157]:
# зафксируем random_state
random_state = 42
# зафксируем max_iter
max_iter = 50

In [158]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=random_state, max_iter = max_iter)
#Обучаем модель
log_reg.fit(X_train, y_train)
y_test_pred = log_reg.predict(X_test)
print('F1 score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1 score: 0.79


In [159]:
#Создаём объект класса RandomForestClassifier
rf_clf = ensemble.RandomForestClassifier(
    random_state=random_state #генератор случайных чисел
)
#Обучаем модель 
rf_clf.fit(X_train, y_train)
y_test_pred = rf_clf.predict(X_test)
print('F1 score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1 score: 0.81


<center> ПАРАМЕТРЫ

In [160]:
#Параметры для логистической регрессии
penalty_choice = ('l2', 'none')
solver_choice = ('lbfgs', 'sag')
C_choice = (0.1, 0.5, 1)

param_grid_lr = {'penalty': penalty_choice , # тип регуляризации
              'solver': solver_choice, # алгоритм оптимизации
               'C': C_choice # уровень силы регурялизации
                }

space_lr = {'penalty': hp.choice('penalty', penalty_choice) , # тип регуляризации
          'solver': hp.choice('solver', solver_choice), # алгоритм оптимизации
          'C': hp.choice('C', C_choice) # уровень силы регурялизации
          }

#Параметры для сдучайного леса
param_grid_rfc = {'min_samples_leaf': list(np.linspace(5, 100, 10, dtype=int)),
              'max_depth': list(np.linspace(1, 30, 10, dtype=int)),
              'criterion':['entropy','gini']
                }

criterion_choice = ('entropy','gini')

space_rfc={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
      'max_depth' : hp.quniform('max_depth', 1, 30, 1),
      'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 100, 10),
      'criterion': hp.choice('criterion', criterion_choice)
      }

### <center> **GridSearchCV**

In [161]:
from sklearn.model_selection import GridSearchCV

In [162]:
#ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ + GridSearchCV

grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=random_state, max_iter=max_iter), 
    param_grid=param_grid_lr, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 1.67 s
Wall time: 23.9 s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


In [163]:
#СЛУЧАЙНЫЕ ЛЕС + GridSearchCV
            
grid_search_tree = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=random_state), 
    param_grid=param_grid_rfc, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_tree.fit(X_train, y_train) 
y_test_pred = grid_search_tree.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_tree.best_params_))

CPU times: total: 8.62 s
Wall time: 3min 26s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 13, 'min_samples_leaf': 5}


### <center> **RANDOMIZEDSEARCHCV**

In [164]:
from sklearn.model_selection import RandomizedSearchCV

In [165]:
#ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ + RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=random_state, max_iter=max_iter), 
    param_distributions=param_grid_lr, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)
%time random_search.fit(X_train, y_train) 
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 1.78 s
Wall time: 21.4 s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}


In [166]:
#СЛУЧАЙНЫЕ ЛЕС + RandomizedSearchCV
            
random_search_tree = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=random_state), 
    param_distributions=param_grid_rfc, 
    cv=5,
    n_iter = 10,
    n_jobs = -1
)  
%time random_search_tree.fit(X_train, y_train) 
y_test_pred = random_search_tree.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_tree.best_params_))

CPU times: total: 1.58 s
Wall time: 12.5 s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'min_samples_leaf': 36, 'max_depth': 20, 'criterion': 'entropy'}


### <center> **HYPEROPT**

ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ + HYPEROPT

In [167]:
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': params['penalty'], 
            'solver': params['solver'],
            'C': params['C']             
            }
  
    # используем эту комбинацию для построения модели
    model_lr = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=max_iter)

    # обучаем модель
    model_lr.fit(X, y)
    score = metrics.f1_score(y, model_lr.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [168]:
%%time

trials_lr = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr, # наша функция 
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_lr, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров - penalty: {}, solver: {}, C: {}".format(
                                            penalty_choice[best['penalty']], solver_choice[best['solver']], C_choice[best['C']]))

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 20/20 [00:31<00:00,  1.57s/trial, best loss: -0.8775698066891684]
Наилучшие значения гиперпараметров - penalty: none, solver: lbfgs, C: 0.5
CPU times: total: 41.5 s
Wall time: 31.5 s


In [169]:
# рассчитаем точность для тестовой выборки
model_lr = linear_model.LogisticRegression(
    random_state=random_state,
    max_iter=max_iter,     
    penalty=penalty_choice[best['penalty']],
    solver=solver_choice[best['solver']],
    C=C_choice[best['C']]
)
model_lr.fit(X_train, y_train)
y_test_pred = model_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ + HYPEROPT + cross validation

In [170]:
def hyperopt_lr_cv(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': params['penalty'], 
             'solver': params['solver'],
             'C': params['C']             
              }
  
    # используем эту комбинацию для построения модели
    model_lr_cv = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=max_iter)

    # обучаем модель
    model_lr_cv.fit(X, y)
    
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model_lr_cv, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [171]:
%%time

trials_lr_cv = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr_cv, # наша функция 
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_lr_cv, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров - penalty: {}, solver: {}, C: {}".format(penalty_choice[best['penalty']], solver_choice[best['solver']], C_choice[best['C']]))

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 20/20 [01:27<00:00,  4.39s/trial, best loss: -0.7818644393726433]
Наилучшие значения гиперпараметров - penalty: l2, solver: lbfgs, C: 0.1
CPU times: total: 40.8 s
Wall time: 1min 27s


In [172]:
# рассчитаем точность для тестовой выборки
model_lr_cv = linear_model.LogisticRegression(
    random_state=random_state,
    max_iter=max_iter,     
    penalty=penalty_choice[best['penalty']],
    solver=solver_choice[best['solver']],
    C=C_choice[best['C']]
)
model_lr_cv.fit(X_train, y_train)
y_test_pred = model_lr_cv.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.79


СЛУЧАЙНЫЕ ЛЕС + HYPEROPT

In [173]:
# зафксируем random_state
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf']),
             'criterion': params['criterion']             
              }
  
    # используем эту комбинацию для построения модели
    model_rfc = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model_rfc.fit(X, y)
    score = metrics.f1_score(y, model_rfc.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [174]:
%%time

trials_rfc = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space_rfc, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_rfc, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров - criterion: {}, max_depth: {}, min_samples_leaf: {}, n_estimators: {}".format(
                                            criterion_choice[best['criterion']], best['max_depth'], best['min_samples_leaf'], best['n_estimators']))

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 20/20 [00:31<00:00,  1.57s/trial, best loss: -0.9022970903522205]
Наилучшие значения гиперпараметров - criterion: gini, max_depth: 19.0, min_samples_leaf: 10.0, n_estimators: 115.0
CPU times: total: 31.2 s
Wall time: 31.4 s


In [175]:
# рассчитаем точность для тестовой выборки
model_rfc = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf']),
    criterion=criterion_choice[best['criterion']]
)
model_rfc.fit(X_train, y_train)
y_test_pred = model_rfc.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


СЛУЧАЙНЫЕ ЛЕС + HYPEROPT + cross validation

In [176]:
# зафксируем random_state
def c(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf']),
             'criterion': params['criterion']             
              }
  
    # используем эту комбинацию для построения модели
    model_rfc_cv = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model_rfc_cv.fit(X, y)
    
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model_rfc_cv, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [177]:
%%time

trials_rfc_cv = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf_cv, # наша функция 
          space=space_rfc, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_rfc_cv, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров - criterion: {}, max_depth: {}, min_samples_leaf: {}, n_estimators: {}".format(
                                            criterion_choice[best['criterion']], best['max_depth'], best['min_samples_leaf'], best['n_estimators']))

100%|██████████| 20/20 [01:10<00:00,  3.54s/trial, best loss: -0.7889298298777034]
Наилучшие значения гиперпараметров - criterion: gini, max_depth: 19.0, min_samples_leaf: 10.0, n_estimators: 115.0
CPU times: total: 30.8 s
Wall time: 1min 10s


In [178]:
# рассчитаем точность для тестовой выборки
model_rfc_cv = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf']),
    criterion=criterion_choice[best['criterion']]
)
model_rfc_cv.fit(X_train, y_train)
y_test_pred = model_rfc_cv.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


### <center> **OPTUNA**

In [179]:
import optuna

ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ + OPTUNA

In [180]:
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', penalty_choice)
  solver = trial.suggest_categorical('solver', solver_choice)
  C = trial.suggest_categorical('C', C_choice)

  # создаем модель
  model_lr = linear_model.LogisticRegression(random_state=random_state,
                                             max_iter=max_iter,
                                             penalty = penalty,
                                             solver = solver,
                                             C = C)
  
  # обучаем модель
  model_lr.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model_lr.predict(X_train))

  return score

In [181]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=20)

[I 2023-06-22 20:45:33,725] A new study created in memory with name: LogisticRegression
[I 2023-06-22 20:45:34,189] Trial 0 finished with value: 0.8483009708737865 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.1}. Best is trial 0 with value: 0.8483009708737865.
[I 2023-06-22 20:45:34,603] Trial 1 finished with value: 0.8708891595615104 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 1}. Best is trial 1 with value: 0.8708891595615104.
[I 2023-06-22 20:45:35,029] Trial 2 finished with value: 0.8682598039215687 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.5}. Best is trial 1 with value: 0.8708891595615104.
[I 2023-06-22 20:45:37,757] Trial 3 finished with value: 0.8690330477356182 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 1}. Best is trial 1 with value: 0.8708891595615104.
[I 2023-06-22 20:45:40,576] Trial 4 finished with value: 0.8661369193154035 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.5}. Best is trial 1 with value: 

CPU times: total: 35.3 s
Wall time: 21.5 s


In [182]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

# рассчитаем точность для тестовой выборки
model_lr = linear_model.LogisticRegression(**study.best_params,random_state=random_state, max_iter=max_iter)
model_lr.fit(X_train, y_train)
y_test_pred = model_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.5}
f1_score на тестовом наборе: 0.78


СЛУЧАЙНЫЕ ЛЕС + OPTUNA

In [183]:
def optuna_rfc(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 1, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 100, 10)
  criterion = trial.suggest_categorical('criterion', criterion_choice)

  # создаем модель
  model_rfc = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          criterion = criterion,
                                          random_state=random_state)
  # обучаем модель
  model_rfc.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model_rfc.predict(X_train))

  return score

In [184]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rfc, n_trials=20)

[I 2023-06-22 20:45:55,629] A new study created in memory with name: RandomForestClassifier
[I 2023-06-22 20:45:57,254] Trial 0 finished with value: 0.7811080835603997 and parameters: {'n_estimators': 162, 'max_depth': 22, 'min_samples_leaf': 85, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7811080835603997.
[I 2023-06-22 20:45:59,675] Trial 1 finished with value: 0.8392802683745044 and parameters: {'n_estimators': 163, 'max_depth': 23, 'min_samples_leaf': 25, 'criterion': 'entropy'}. Best is trial 1 with value: 0.8392802683745044.
[I 2023-06-22 20:46:00,682] Trial 2 finished with value: 0.7897590361445783 and parameters: {'n_estimators': 108, 'max_depth': 20, 'min_samples_leaf': 75, 'criterion': 'gini'}. Best is trial 1 with value: 0.8392802683745044.
[I 2023-06-22 20:46:02,203] Trial 3 finished with value: 0.8204031765424558 and parameters: {'n_estimators': 113, 'max_depth': 17, 'min_samples_leaf': 35, 'criterion': 'entropy'}. Best is trial 1 with value: 0.8392802683745044.

CPU times: total: 50.5 s
Wall time: 50.7 s


In [185]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

# рассчитаем точность для тестовой выборки
model_rfc = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state)
model_rfc.fit(X_train, y_train)
y_test_pred = model_rfc.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'n_estimators': 195, 'max_depth': 30, 'min_samples_leaf': 5, 'criterion': 'entropy'}
f1_score на тестовом наборе: 0.82
